In [1]:
!conda install -n tfm -c conda-forge pytorch-lightning --yes

Solving environment: done

# All requested packages already installed.

Retrieving notices: ...working... done


In [2]:
# To help preventing shared maemory errors
!ulimit -n 64000
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

In [3]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pickle
import numpy as np
import pandas as pd
import random
from Utils.pretrainedGloVe import pretrainedWordEmeddings
from DataLoader.swde_dataLoader import swde_data_test, collate_fn_test
from Model.SimpDOM_model import SeqModel
from Prediction.test_step import main as get_predictions
from Utils.logger import logger

In [4]:
# Model Configurations

datapath = './data'
random.seed(7)
device = 'cpu'

n_workers=0 # Must be 0 to prevent shared memory issues!
n_gpus=0
char_emb_dim = 16
char_hid_dim = 100
char_emb_dropout = 0.1

tag_emb_dim = 16
tag_hid_dim = 30

leaf_emb_dim = 30
pos_emb_dim = 20
word_emb_filename= '{}/glove.6B.100d.txt'.format(datapath)

train_websites = ['auto-aol','auto-yahoo','auto-motortrend','auto-autobytel', 'auto-msn', ]
val_websites = ['auto-aol','auto-yahoo']
attributes = ['model', 'price', 'engine', 'fuel_economy']
n_classes = len(attributes)+1
class_weights = [1,100,100,100,100]

In [ ]:
#Data Loading

logger.info(f'Loading {datapath}/English_charDict.pkl')
charDict = pickle.load(open('{}/English_charDict.pkl'.format(datapath),'rb'))
logger.info(f'Loading {datapath}/HTMLTagDict.pkl')
tagDict = pickle.load(open('{}/HTMLTagDict.pkl'.format(datapath),'rb'))
logger.info(f'Char dictionary length: {len(charDict)}, Tag dictionary length: {len(tagDict)}')

logger.info(f'Loading pretrained word emeddings from: {datapath}/glove.6B.100d.txt')
WordEmeddings = pretrainedWordEmeddings('{}/glove.6B.100d.txt'.format(datapath))
logger.info(f'Start generating test dataset')
test_dataset = DataLoader(dataset = swde_data_test(val_websites, datapath, charDict, \
                                  tagDict, n_gpus, WordEmeddings), num_workers=n_workers, \
                                  batch_size=32, shuffle=False, collate_fn = collate_fn_test)

14:40:32 INFO (2422149600:3): Loading ./data/English_charDict.pkl
14:40:32 INFO (2422149600:5): Loading ./data/HTMLTagDict.pkl
14:40:32 INFO (2422149600:7): Char dictionary length: 85, Tag dictionary length: 39
14:40:32 INFO (2422149600:9): Loading pretrained word emeddings from: ./data/glove.6B.100d.txt
14:40:36 INFO (2422149600:11): Start generating test dataset
14:40:36 INFO (swde_dataLoader:236): Start loading data set for websites: ['auto-aol', 'auto-yahoo']


pretrained word vectors loaded - 400000


Web site:   0%|          | 0/2 [00:00<?, ?it/s]

Web pages:   0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
logger.info('Instantiating the Model checkpoint')
checkpoint_callback = ModelCheckpoint(
    filename='./data/weights',
    save_top_k=1,
    save_last = True,
    verbose=True,
    monitor='val_loss',
    mode='min'
)

config = {
    'out_dim': n_classes,
    'train_websites': train_websites,
    'val_websites': val_websites,
    'datapath': datapath,
    'n_workers': n_workers,
    'charDict' : charDict,
    'char_emb_dim' : char_emb_dim,
    'char_hid_dim' : char_hid_dim,
    'char_emb_dropout' : char_emb_dropout,
    'tagDict': tagDict,
    'tag_emb_dim': tag_emb_dim,
    'tag_hid_dim': tag_hid_dim,
    'leaf_emb_dim': leaf_emb_dim,
    'pos_emb_dim': pos_emb_dim,
    'attributes': attributes,
    'n_gpus' : n_gpus,
    'class_weights':class_weights,
    'word_emb_filename': word_emb_filename
}
logger.info(f'Model config: {config}')

In [ ]:
logger.info('Loading the Sequential model from Checkpoint')
model = SeqModel.load_from_checkpoint(f'{datapath}/weights.ckpt', config=config)
model = model.eval()
model = model.to(device)

In [ ]:
logger.info('Generating model predictions')
df = get_predictions(test_dataset, model, device, 0.6)

In [ ]:
from Prediction.PRSummary import cal_PR_summary
avg_prf1_dict = cal_PR_summary(df, n_classes)

from Prediction.WebsiteLevel_PR_Generator import cal_PR_summary as websiteLevel_cal_PR_summary
pr_summary_df, pr_results_df = websiteLevel_cal_PR_summary(df, n_classes)
print(pr_results_df)